# Business Understanding
By [THE TIMES OF INDIA](https://timesofindia.indiatimes.com/business/india-business/india-becomes-third-largest-startup-ecosystem-in-the-world/articleshow/85871428.cms), India has emerged as the third largest startup ecosystem in the world after US and China. Following this, our team aims to strategically enter the Indian Startup Ecosystem by leveraging data-driven insights to identify high-potential opportunities. Through comprehensive research and analysis, we seek to gain insight into funding received by startups in India from 2018 to 2021.

#### Hypothesis
**Null Hypothesis (Ho):** The sector of a startup has no significant influence on the funding it receives.<br>

**Alternative Hypothesis (Ha):** The sector of a startup has significant influence on the funding it receives.

# Data Understanding
This data provides information into amount of money startups received from 2018 to 2021, the sector of startups, headquaters, what a startup do, the year of establishment, startup name, investors, and stage.<br>

`Feature Description`:
- **Company_Brand:** Name of startup
- **Founded:** Year of establishment
- **HeadQuater:** Location of startup Headquater
- **Sector:** Sector or industry of startup
- **What_it_does:** what the startup does
- **Founders:** Name od founder
- **Investors:** Name of investor
- **Amount:** Amount of investment in USD and INR
- **Statge:** Phase of development (eg. Ideation Stage, Pre-Seed Stage, Seed Stage, Early Stage (Series A, B, etc.))
- **Year:** Year startup received funding

#### Analytical Questions
1. How is funding spread across the years?
2. What are the dominant sectors within the Indian startup ecosystem across the years?
3. Are there any emerging sectors that have shown a significant increase in funding year over year?
4. Where in India could be considered the surviving grounds for startups?
5. How does the startup's location influence its funding and growth opportunities?
6. Is there a relationship between what a startup does and the funding it receives?
7. Is there a correlation between the year a startup received funding and the amount of funding it received?
8. Which cities or regions have the highest concentration of funded startups?

### Import libraries

In [1]:
# Import necesary libraries and packages
import pyodbc
from dotenv import dotenv_values #import the dotenv_values function from the dotenv package
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Load Environment Variables and Create SQL Server Connection

In [2]:
# Load environment variables from .env file
environment_variables = dotenv_values('.env')
# Access login credentials from  the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("UID")
password = environment_variables.get("PWD")

In [3]:
#  Create Connection connection string
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# connect to the server using pyodbc
connection = pyodbc.connect(connection_string)

### Load Datasets 

In [4]:
# Write querry to retrieve tables from database
query1 = "Select * from dbo.LP1_startup_funding2020"
query2 = "Select * from dbo.LP1_startup_funding2021"

# Retrieve dataset from database with connection created
df_2020 = pd.read_sql(query1, connection)
df_2021 = pd.read_sql(query2, connection)

# Load CSV files
df_2018 = pd.read_csv('Data\startup_funding2018.csv')
df_2019 = pd.read_csv('Data\startup_funding2019.csv')

## Exploring data quality and characteristics

#### 2020 Dataset Exploration and Preparation

In [5]:
# Preview dataframe
df_2020.head(3)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,None
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,None
2,PadCare Labs,2018.0,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,Ajinkya Dhariya,Venture Center,NaN,Pre-seed,None


In [6]:
# Check characteristics of dataframe
print(df_2020.info(), "\n====================== Null Value Percentage ==========================")
# Check for null values
print(df_2020.isna().mean().mul(100), "\n======================= Duplicated rows =========================")
# Check for duplicate
df_2020.loc[df_2020.duplicated()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1055 non-null   object 
 1   Founded        842 non-null    float64
 2   HeadQuarter    961 non-null    object 
 3   Sector         1042 non-null   object 
 4   What_it_does   1055 non-null   object 
 5   Founders       1043 non-null   object 
 6   Investor       1017 non-null   object 
 7   Amount         801 non-null    float64
 8   Stage          591 non-null    object 
 9   column10       2 non-null      object 
dtypes: float64(2), object(8)
memory usage: 82.5+ KB
None 
====================== Null Value Percentage ==========================
Company_Brand     0.000000
Founded          20.189573
HeadQuarter       8.909953
Sector            1.232227
What_it_does      0.000000
Founders          1.137441
Investor          3.601896
Amount           24.075829
Stage            4

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10
145,Krimanshi,2015.0,Jodhpur,Biotechnology company,Krimanshi aims to increase rural income by imp...,Nikhil Bohra,"Rajasthan Venture Capital Fund, AIM Smart City",600000.0,Seed,None
205,Nykaa,2012.0,Mumbai,Cosmetics,Nykaa is an online marketplace for different b...,Falguni Nayar,"Alia Bhatt, Katrina Kaif",NaN,None,None
362,Byju’s,2011.0,Bangalore,EdTech,An Indian educational technology and online tu...,Byju Raveendran,"Owl Ventures, Tiger Global Management",500000000.0,None,None


Drop Duplicates

In [7]:
# Drop duplicated rows from DataFrame
df_2020.drop_duplicates(keep = "first", inplace = True)

Column10 has 99.8% null values so we drop it

In [8]:
# Drop columns10
df_2020.drop(columns = ["column10"], inplace = True)
# Preview remaining columns 
df_2020.columns

Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Amount', 'Stage'],
      dtype='object')

#### 2021 Dataset Exploration and Preparation

In [9]:
# Preview dataframe
df_2021.head(3)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital","$30,000,000",Series D


In [10]:
# Check characteristics of dataframe
print(df_2021.info(), "\n====================== Null Value Percentage ======================")
# Check for null values
print(df_2021.isna().mean().mul(100), "\n====================== Duplicated rows ======================")
# Check for duplicates
df_2021.loc[df_2021.duplicated()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1209 non-null   object 
 1   Founded        1208 non-null   float64
 2   HeadQuarter    1208 non-null   object 
 3   Sector         1209 non-null   object 
 4   What_it_does   1209 non-null   object 
 5   Founders       1205 non-null   object 
 6   Investor       1147 non-null   object 
 7   Amount         1206 non-null   object 
 8   Stage          781 non-null    object 
dtypes: float64(1), object(8)
memory usage: 85.1+ KB
None 
====================== Null Value Percentage ======================
Company_Brand     0.000000
Founded           0.082713
HeadQuarter       0.082713
Sector            0.000000
What_it_does      0.000000
Founders          0.330852
Investor          5.128205
Amount            0.248139
Stage            35.401158
dtype: float64 
====================== D

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
107,Curefoods,2020.0,Bangalore,Food & Beverages,Healthy & nutritious foods and cold pressed ju...,Ankit Nagori,"Iron Pillar, Nordstar, Binny Bansal",$13000000,None
109,Bewakoof,2012.0,Mumbai,Apparel & Fashion,Bewakoof is a lifestyle fashion brand that mak...,Prabhkiran Singh,InvestCorp,$8000000,None
111,FanPlay,2020.0,Computer Games,Computer Games,A real money game app specializing in trivia g...,YC W21,"Pritesh Kumar, Bharat Gupta",Upsparks,$1200000
117,Advantage Club,2014.0,Mumbai,HRTech,Advantage Club is India's largest employee eng...,"Sourabh Deorah, Smiti Bhatt Deorah","Y Combinator, Broom Ventures, Kunal Shah",$1700000,None
119,Ruptok,2020.0,New Delhi,FinTech,Ruptok fintech Pvt. Ltd. is an online gold loa...,Ankur Gupta,Eclear Leasing,$1000000,None
243,Trinkerr,2021.0,Bangalore,Capital Markets,Trinkerr is India's first social trading platf...,"Manvendra Singh, Gaurav Agarwal",Accel India,$6600000,Series A
244,Zorro,2021.0,Gurugram,Social network,Pseudonymous social network platform,"Jasveer Singh, Abhishek Asthana, Deepak Kumar","Vijay Shekhar Sharma, Ritesh Agarwal, Ankiti Bose",$32000000,Seed
245,Ultraviolette,2021.0,Bangalore,Automotive,Create and Inspire the future of sustainable u...,"Subramaniam Narayan, Niraj Rajmohan","TVS Motor, Zoho",$150000000,Series C
246,NephroPlus,2009.0,Hyderabad,Hospital & Health Care,A vision and passion of redefining healthcare ...,Vikram Vuppala,IIFL Asset Management,$24000000,Series E
247,Unremot,2020.0,Bangalore,Information Technology & Services,Unremot is a personal office for consultants!,Shiju Radhakrishnan,Inflection Point Ventures,$700000,Seed


Drop Duplicates

In [11]:
# Drop duplicated rows
df_2021.drop_duplicates(keep = "first", inplace = True)

#### 2018 Dataset Exploration and Preparation


In [12]:
# Preview dataframe
df_2018.head(3)

,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India


In [13]:
# Check characteristics of dataframe
print(df_2018.info(), "\n====================== Null Value Percentage ======================")
# Check for null values
print(df_2018.isna().mean().mul(100), "\n====================== Duplicated rows ======================")
# Check for duplicates
df_2018.loc[df_2018.duplicated()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company Name   526 non-null    object
 1   Industry       526 non-null    object
 2   Round/Series   526 non-null    object
 3   Amount         526 non-null    object
 4   Location       526 non-null    object
 5   About Company  526 non-null    object
dtypes: object(6)
memory usage: 24.8+ KB
None 
====================== Null Value Percentage ======================
Company Name     0.0
Industry         0.0
Round/Series     0.0
Amount           0.0
Location         0.0
About Company    0.0
dtype: float64 
====================== Duplicated rows ======================


,Company Name,Industry,Round/Series,Amount,Location,About Company
348,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."


Drop Duplicates

In [14]:
# Drop duplicated rows from DataFrame
df_2018.drop_duplicates(keep = "first", inplace = True)

#### 2019 Dataset Exploration and Preparation


In [15]:
# Preview dataframe
df_2019.head(3)

,Company/Brand,Founded,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage
0,Bombay Shaving,NaN,NaN,Ecommerce,Provides a range of male grooming products,Shantanu Deshpande,Sixth Sense Ventures,"$6,300,000",NaN
1,Ruangguru,2014.0,Mumbai,Edtech,A learning platform that provides topic-based ...,"Adamas Belva Syah Devara, Iman Usman.",General Atlantic,"$150,000,000",Series C
2,Eduisfun,NaN,Mumbai,Edtech,It aims to make learning fun via games.,Jatin Solanki,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey","$28,000,000",Fresh funding


In [16]:
# Check characteristics of dataframe
print(df_2019.info(), "\n====================== Null Value Percentage ======================")
# Check for null values
print(df_2019.isna().mean().mul(100), "\n====================== Duplicated rows ======================")
# Check for duplicate
df_2019.loc[df_2019.duplicated()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company/Brand  89 non-null     object 
 1   Founded        60 non-null     float64
 2   HeadQuarter    70 non-null     object 
 3   Sector         84 non-null     object 
 4   What it does   89 non-null     object 
 5   Founders       86 non-null     object 
 6   Investor       89 non-null     object 
 7   Amount($)      89 non-null     object 
 8   Stage          43 non-null     object 
dtypes: float64(1), object(8)
memory usage: 6.4+ KB
None 
====================== Null Value Percentage ======================
Company/Brand     0.000000
Founded          32.584270
HeadQuarter      21.348315
Sector            5.617978
What it does      0.000000
Founders          3.370787
Investor          0.000000
Amount($)         0.000000
Stage            51.685393
dtype: float64 
====================== Duplic

,Company/Brand,Founded,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage


### Add year_funded column to each dataset

In [17]:
# Add year column to dataframe
df_2020["Year_Funded"] = 2020
# Convert data type to datetime format
df_2020["Year_Funded"] = pd.to_datetime(df_2020["Year_Funded"], format = "%Y")

# Add year column to dataframe
df_2021["Year_Funded"] = 2021
# Convert data type to datetime format
df_2021["Year_Funded"] = pd.to_datetime(df_2021["Year_Funded"], format = "%Y")

# Add year column to dataframe
df_2019["Year_Funded"] = 2019
# Convert data type to datetime format
df_2019["Year_Funded"] = pd.to_datetime(df_2019["Year_Funded"], format = "%Y")

# Add year column to dataframe
df_2018["Year_Funded"] = 2018
# Convert data type to datetime format
df_2018["Year_Funded"] = pd.to_datetime(df_2018["Year_Funded"], format = "%Y")

# Preview dataframe
df_2020.head(3)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year_Funded
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,2020-01-01
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,2020-01-01
2,PadCare Labs,2018.0,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,Ajinkya Dhariya,Venture Center,NaN,Pre-seed,2020-01-01


#### Establish uniformity in column names

In [18]:
# Rename columns
df_2019.rename(columns={'Company/Brand':'Company_Brand','Amount($)':'Amount', 'What it does': 'What_it_does'}, inplace=True)

df_2018.rename(columns={'Company Name':'Company_Brand','Industry':'Sector','Round/Series':'Stage','Location':'HeadQuarter','About Company':'What_it_does'}, inplace=True)

In [19]:
def rename_column(df):
    """
    This function takes in a dataframe and renames the column names to lower case
    """
    df.columns = [col_name.lower() for col_name in df.columns]
    return df

# Apply function to DataFrames
df_2018.pipe(rename_column)
df_2019.pipe(rename_column)
df_2020.pipe(rename_column)
df_2021.pipe(rename_column)

# Preview column names
df_2021.columns

Index(['company_brand', 'founded', 'headquarter', 'sector', 'what_it_does',
       'founders', 'investor', 'amount', 'stage', 'year_funded'],
      dtype='object')

### Verify if column name is standardised

In [20]:
# Compare if column names are uniform
print(df_2018.columns)
print(df_2019.columns)
print(df_2020.columns)
print(df_2021.columns)

Index(['company_brand', 'sector', 'stage', 'amount', 'headquarter',
       'what_it_does', 'year_funded'],
      dtype='object')
Index(['company_brand', 'founded', 'headquarter', 'sector', 'what_it_does',
       'founders', 'investor', 'amount', 'stage', 'year_funded'],
      dtype='object')
Index(['company_brand', 'founded', 'headquarter', 'sector', 'what_it_does',
       'founders', 'investor', 'amount', 'stage', 'year_funded'],
      dtype='object')
Index(['company_brand', 'founded', 'headquarter', 'sector', 'what_it_does',
       'founders', 'investor', 'amount', 'stage', 'year_funded'],
      dtype='object')


## Concatenate Datasets

In [21]:
# Combine all four datasets into a single dataframe
df_combined = pd.concat([df_2020, df_2021, df_2018, df_2019], axis = 0)
# Preview combined dataframe
df_combined.head(3)

,company_brand,founded,headquarter,sector,what_it_does,founders,investor,amount,stage,year_funded
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,2020-01-01
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,2020-01-01
2,PadCare Labs,2018.0,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,Ajinkya Dhariya,Venture Center,NaN,Pre-seed,2020-01-01


In [22]:
# Check characteristics of dataframe
print(df_combined.info(), "\n====================== Null Value Percentage ======================")
# Check for null values
print(df_combined.isna().mean().mul(100), "\n====================== Duplicated rows ======================")
# Check for duplicated rows
df_combined.loc[df_combined.duplicated()]

<class 'pandas.core.frame.DataFrame'>
Index: 2856 entries, 0 to 88
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   company_brand  2856 non-null   object        
 1   founded        2088 non-null   float64       
 2   headquarter    2742 non-null   object        
 3   sector         2838 non-null   object        
 4   what_it_does   2856 non-null   object        
 5   founders       2312 non-null   object        
 6   investor       2232 non-null   object        
 7   amount         2600 non-null   object        
 8   stage          1927 non-null   object        
 9   year_funded    2856 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 245.4+ KB
None 
====================== Null Value Percentage ======================
company_brand     0.000000
founded          26.890756
headquarter       3.991597
sector            0.630252
what_it_does      0.000000
founde

,company_brand,founded,headquarter,sector,what_it_does,founders,investor,amount,stage,year_funded


# Data Preparation

Drop columns that are not present throught the four datasets

In [23]:
# Drop columns
df_combined.drop(columns = ["founded", "founders", "investor"], inplace = True)
# Verify if columns are dropped
df_combined.head(3)

,company_brand,headquarter,sector,what_it_does,amount,stage,year_funded
0,Aqgromalin,Chennai,AgriTech,Cultivating Ideas for Profit,200000.0,None,2020-01-01
1,Krayonnz,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,100000.0,Pre-seed,2020-01-01
2,PadCare Labs,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,NaN,Pre-seed,2020-01-01
3,NCOME,New Delhi,Escrow,Escrow-as-a-service platform,400000.0,None,2020-01-01
4,Gramophone,Indore,AgriTech,Gramophone is an AgTech platform enabling acce...,340000.0,None,2020-01-01


## Column Cleaning
### `Column: amount`

In [24]:
# Preview unique entries in the amount column
df_combined["amount"].unique()[:40]

array([200000.0, 100000.0, nan, 400000.0, 340000.0, 600000.0, 45000000.0,
       1000000.0, 2000000.0, 1200000.0, 660000000.0, 120000.0, 7500000.0,
       5000000.0, 500000.0, 3000000.0, 10000000.0, 145000000.0,
       100000000.0, 21000000.0, 4000000.0, 20000000.0, 560000.0, 275000.0,
       4500000.0, 15000000.0, 390000000.0, 7000000.0, 5100000.0,
       700000000.0, 2300000.0, 700000.0, 19000000.0, 9000000.0,
       40000000.0, 750000.0, 1500000.0, 7800000.0, 50000000.0, 80000000.0],
      dtype=object)

Amount and Stage columns have some of their values interchanged and need to be cleaned

In [25]:
#Preview unique entries in the stage column
df_combined["stage"].unique()[30:-15]

array(['Pre series A1', 'Series E2', 'Pre series A', 'Seed Round',
       'Bridge Round', 'Pre seed round', 'Pre series B', 'Pre series C',
       'Seed Investment', 'Series D1', 'Mid series', 'Series C, D',
       'Seed funding', '$1200000', 'Seed+', 'Series F2', 'Series A+',
       'Series G', 'Series B3', 'PE', 'Series F1', 'Pre-series A1',
       '$300000', 'Early seed', '$6000000', '$1000000', 'Seies A',
       'Series A2', 'Series I', 'Angel', 'Private Equity',
       'Venture - Series Unknown'], dtype=object)

In [26]:
df_combined.loc[df_combined["stage"] == "$1000000"]

,company_brand,headquarter,sector,what_it_does,amount,stage,year_funded
677,Saarthi Pedagogy,Ahmadabad,EdTech,"India's fastest growing Pedagogy company, serv...","JITO Angel Network, LetsVenture",$1000000,2021-01-01


In [27]:
# Put values in their appropriate columns
df_combined.at[677, "amount"], df_combined.at[677, "stage"] = 1000000, np.NAN
df_combined.at[545, "amount"], df_combined.at[545, "stage"] = np.NAN, "Pre-series A"
df_combined.at[551, "amount"], df_combined.at[551, "stage"] = 300000, np.NAN
df_combined.at[538, "amount"], df_combined.at[538, "stage"] = 300000, np.NAN
df_combined.at[242, "amount"], df_combined.at[242, "stage"] = np.NAN, "Seed"
df_combined.at[257, "amount"], df_combined.at[257, "stage"] = np.NAN, "Seed"
df_combined.at[1148, "amount"], df_combined.at[1148, "stage"] = np.NAN, "Seed"
df_combined.at[98, "amount"], df_combined.at[98, "stage"] = 1200000, np.NAN
df_combined.at[674, "amount"], df_combined.at[674, "stage"] = 6000000, np.NAN


Remove currency signs, convert INR (₹) to USD ($), and convert datatype to float

In [28]:
# Clean Amount colum and convert Indian Rupee to USD currency
def clean_amount(value):
    """ 
    Removes "$", and "₹"  and converts column to float
     """
    try:
        value = str(value)
        # Remove commas
        value = value.replace(",", "")
        # Check if the amount is in INR and convert to USD assuming 1 USD = 70 INR
        if "₹" in value:
            valuet = value.replace("₹", "")
            return round(float(value) / 70, 2)
        # Check if the amount is in USD
        elif "$" in value:
            value = value.replace("$", "")
            return round(float(value), 2) 
        # If no currency symbol, assume it's already in USD
        else:
            return round(float(value), 2)
    except ValueError:
        # For non-numeric entries, return NaN
        return np.NAN

# Apply the clean_amount function to the 'amount' column
df_combined["amount"] = df_combined["amount"].apply(clean_amount)

# Preview dataframe
df_combined["amount"].unique()[:35]

array([2.00e+05, 1.00e+05,      nan, 4.00e+05, 3.40e+05, 6.00e+05,
       4.50e+07, 1.00e+06, 2.00e+06, 1.20e+06, 6.60e+08, 1.20e+05,
       7.50e+06, 5.00e+06, 5.00e+05, 3.00e+06, 1.00e+07, 1.45e+08,
       1.00e+08, 2.10e+07, 4.00e+06, 2.00e+07, 5.60e+05, 2.75e+05,
       4.50e+06, 1.50e+07, 3.90e+08, 7.00e+06, 5.10e+06, 7.00e+08,
       2.30e+06, 7.00e+05, 1.90e+07, 9.00e+06, 4.00e+07])

Fill null values with the median<br>**NB**: Mean is affected by outliers and following that the alternative is to employ the median approach

In [29]:
# Fill null values with the median
df_combined.fillna({"amount": df_combined["amount"].median()}, inplace = True)
# Verify filled null values
df_combined.isna().sum()

company_brand      0
headquarter      114
sector            18
what_it_does       0
amount             0
stage            933
year_funded        0
dtype: int64

### `Column: headquarter`

In [30]:
# Preview unique values in the headquarter column
df_combined["headquarter"].unique()[80:-50]

array(['Rajsamand', 'Ranchi', 'Faridabad, Haryana', 'Computer Games',
       'Vadodara', 'Food & Beverages', 'Pharmaceuticals\t#REF!',
       'Gurugram\t#REF!', 'Mohali', 'Powai', 'Ghaziabad', 'Nagpur',
       'West Bengal', 'Samsitpur', 'Lucknow', 'Telangana', 'Silvassa',
       'Thiruvananthapuram', 'Faridabad', 'Roorkee', 'Ambernath',
       'Panchkula', 'Surat', 'Mangalore', 'Telugana', 'Bhubaneswar',
       'Kottayam', 'Beijing', 'Panaji', 'Satara', 'Orissia', 'Santra',
       'Mountain View, CA', 'Trivandrum', 'Jharkhand', 'Bhilwara',
       'Guwahati', 'Online Media\t#REF!', 'London',
       'Information Technology & Services', 'The Nilgiris', 'Gandhinagar',
       'Bangalore, Karnataka, India', 'Mumbai, Maharashtra, India'],
      dtype=object)

headquarter, sector, and what it does columns have some of their values interchanged and needs to be cleaned

In [31]:
df_combined.loc[df_combined["headquarter"] == "Information Technology & Services"]

,company_brand,headquarter,sector,what_it_does,amount,stage,year_funded
1176,Peak,Information Technology & Services,"Manchester, Greater Manchester",Peak helps the world's smartest companies put ...,75000000.0,Series C,2021-01-01


In [32]:
# Put values in their appropriate columns
df_combined.at[98, "headquarter"] = "None"
df_combined.at[241, "headquarter"], df_combined.at[241, "sector"] = "Hauz Khas", "Food & Beverages"
df_combined.at[242, "what_it_does"] = df_combined.at[242, "sector"]
df_combined.at[242, "headquarter"], df_combined.at[242, "sector"] = "None", "Pharmaceuticals"
df_combined.at[257, "what_it_does"] = df_combined.at[257, "sector"]
df_combined.at[257, "headquarter"], df_combined.at[257, "sector"] = "Gurugram", "None"
df_combined.at[1100, "what_it_does"] = df_combined.at[1100, "sector"]
df_combined.at[1100, "headquarter"], df_combined.at[1100, "sector"] = "None", "Online Media"
df_combined.at[1176, "headquarter"], df_combined.at[1176, "sector"] = "Manchester, Greater Manchester", "Information Technology & Services"

Correct spelling mistakes in the headquarter column

In [33]:
def clean_column(value):
    """
    Corrects misspelled values, splits string at the comma, and takes the first part
    """
    replacement = {"New Delhi": "Delhi", "New New Delhi": "Delhi", "San Franciscao": "San Francisco", "San Francisco Bay Area": "San Francisco",
                   "Bangaldesh": "Bangladesh", "Milano": "Milan", "Newcastle Upon Tyne": "Newcastle", "Hyderebad": "Hyderabad", "Banglore": "Bengaluru",
                   "Bangalore": "Bengaluru", "Santra": "Santa", "Orissa": "Odisha", "Kormangala": "Koramangala", "Cochin": "Kochi", "Orissia": "Odisha",
                   "Thiruvananthapuram": "Trivandrum", "Samsitpur": "Samastipur", "Telugana": "Telangana", "Gurgaon": "Gurugram", "Rajastan": "Rajasthan",
                   "Uttar pradesh": "Uttar Pradesh", "Ahmadabad": "Ahmedabad"}
    
    value = str(value)
    # Condition ensures function processes non-None values
    if value is not None:
        for old_value, new_value in replacement.items():
            # Split comma separated values and extract the first value
            value = value.split(",")[0]
            # Replace mispelled values with corrected spelled values
            value = value.replace(old_value, new_value)
        return value

# Apply function to column
df_combined["headquarter"] = df_combined["headquarter"].apply(clean_column)

Preview headquarter column

In [34]:
# Replace "nan" with np.NAN for it to be recognised as a null value
df_combined["headquarter"].replace("nan", np.NAN, inplace = True)
df_combined["headquarter"].replace("None", np.NAN, inplace = True)
# Preview column
df_combined["headquarter"].unique()[:20]

array(['Chennai', 'Bengaluru', 'Pune', 'Delhi', 'Indore', 'Hyderabad',
       'Gurugram', 'Belgaum', 'Noida', 'Mumbai', 'Andheri', 'Jaipur',
       'Ahmedabad', 'Kolkata', 'Tirunelveli', 'Thane', nan, 'Singapore',
       'Haryana', 'Kerala'], dtype=object)

Categorising headquarter based on location

In [35]:
def city_names(value):
    """
    Replaces sub cities with their major cities
    """
    cities = {"Bengaluru": ["Bengaluru City", "Koramangala"], "Delhi": ["Kalkaji", "Hauz Khas", "Azadpur"], "California": ["San Francisco", "Mountain View", "San Ramon", "Irvine"],
            "Tamil Nadu": ["Chennai", "Tirunelveli", "Guindy", "Mylapore", "Kalpakkam"], "Maharashtra": ["Andheri", "Thane", "Worli"],
            "Uttar Pradesh": ["Ghaziabad", "Uttar", "Lucknow", "Kanpur", "Noida"], "Rajasthan": ["Alwar", "Rajasthan"], "Mumbai": ["Powai", "Andheri"],
            "Kerala": ["Kochi", "Alleppey", "Kannur", "Ernakulam", "Trivandrum"], "Margão": ["Goa"], "Anand": ["Gujarat"], "Hyderabad": ["Telangana"],
            "Bihar": ["Samastipur", "Bihar"], "Rajasthan": ["Jodhpur"], "Mumbai": ["Powai", "Andheri"], "Gurgaon": ["Gurugram"]}
    
    # Put headquarter into categories
    for major_city, sub_city in cities.items():
        if value in sub_city:
            value = major_city
    return value

# Apply function to column
df_combined["headquarter"] = df_combined["headquarter"].apply(city_names)
# Preview column
df_combined["headquarter"].unique()

array(['Tamil Nadu', 'Bengaluru', 'Pune', 'Delhi', 'Indore', 'Hyderabad',
       'Gurgaon', 'Belgaum', 'Uttar Pradesh', 'Mumbai', 'Maharashtra',
       'Jaipur', 'Ahmedabad', 'Kolkata', nan, 'Singapore', 'Haryana',
       'Kerala', 'Rajasthan', 'Anand', 'Frisco', 'California',
       'Dhingsara', 'New York', 'Patna', 'Paris', 'Plano', 'Sydney',
       'Bangladesh', 'London', 'Milan', 'Palmwoods', 'France', 'Bihar',
       'Tumkur', 'Newcastle', 'Shanghai', 'Jiaxing', 'Ludhiana',
       'Dehradun', 'Tangerang', 'Berlin', 'Seattle', 'Riyadh', 'Seoul',
       'Bangkok', 'Chandigarh', 'Warangal', 'Odisha', 'Margão', 'Bhopal',
       'Coimbatore', 'Small Towns', 'Rajsamand', 'Ranchi', 'Faridabad',
       'Vadodara', 'Mohali', 'Nagpur', 'West Bengal', 'Silvassa',
       'Roorkee', 'Ambernath', 'Panchkula', 'Surat', 'Mangalore',
       'Bhubaneswar', 'Kottayam', 'Beijing', 'Panaji', 'Satara', 'Santa',
       'Jharkhand', 'Bhilwara', 'Guwahati', 'Manchester', 'The Nilgiris',
       'Gandhinaga

### `Column: stage`

In [36]:
df_combined["stage"].unique()

array([None, 'Pre-seed', 'Seed', 'Pre-series A', 'Pre-series', 'Series C',
       'Series A', 'Series B', 'Debt', 'Pre-series C', 'Pre-series B',
       nan, 'Series E', 'Bridge', 'Series D', 'Series B2', 'Series F',
       'Pre- series A', 'Edge', 'Series H', 'Pre-Series B', 'Seed A',
       'Series A-1', 'Seed Funding', 'Pre-Seed', 'Seed round',
       'Pre-seed Round', 'Seed Round & Series A', 'Pre Series A',
       'Pre seed Round', 'Angel Round', 'Pre series A1', 'Pre series A',
       'Seed Round', 'Bridge Round', 'Pre seed round', 'Pre series B',
       'Pre series C', 'Seed Investment', 'Series D1', 'Mid series',
       'Series C, D', 'Seed funding', 'Seed+', 'Series F2', 'Series A+',
       'Series G', 'Series B3', 'PE', 'Series F1', 'Pre-series A1',
       'Early seed', 'Seies A', 'Series A2', 'Series I', 'Angel',
       'Private Equity', 'Venture - Series Unknown', 'Grant',
       'Debt Financing', 'Post-IPO Debt', 'Corporate Round',
       'Undisclosed',
       'https://doc

Categorise Stages of funding according to the Indian Startup Ecosystem [Funding Guide](https://www.startupindia.gov.in/content/sih/en/funding.html) 

| Stages Classification | Description | Stages of funding per Data given |
|----------|-----------|-----------|
| Ideation | Brainstorming and developing business concepts, defining value propositions, and outlining plans | Pre-seed, Pre-Seed, Pre seed Round, Early seed, Pre-seed Round, Pre seed round |
| Validation | Validating the business model, product-market fit, and scalability through research and feedback | Angel, Seed, Seed A, Seed Funding, Seed round, Seed Round, Seed Round & Series A, Seed Investment, Angel Round, Seed fund, Seed funding, Seed+, Grant, Corporate Round |
| Early Traction | Gaining initial traction, attracting early adopters, and refining based on feedback | Series A, Series A-1, Series A+, Series A2, Seies A, Pre-Series A, Pre-series A, Pre series A1, Pre-series A1, Post series A, Pre Series A, Pre series A, Pre-series, Pre- series A, Venture - Series Unknown |
| Scaling | Expanding operations, customer base, and market reach for rapid growth | Series B, Series B2, Series B3, Series C, Series C, D, Series D, Series D1, Series E, Series F, Series F1, Series F2, Series G, Series H, Series I, Mid series, Pre-series B, Pre-Series B, Pre series B, Pre-series C, Pre series C, Bridge, Edge, Series B+, Post-IPO Equity, Debt, Debt Financing, Private Equity (PE), PE, Private Equity |
| Exit Options | Considering exit strategies such as mergers, acquisitions, or IPOs | Post-IPO Debt, Bridge Round |
| Others | Stages with unknown class | Fresh funding, Funding Round, Non-equity Assistance, Secondary Market, Undisclosed, https://docs.google.com/spreadsheets/d/1x9ziNeaz6auNChIHnMI8U6kS7knTr3byy_YBGfQaoUA/edit#gid=1861303593 |


Group stages according to the Indian Startup Ecosystem funding stage

In [37]:
def stage_classification(value):
    """
    Groups stages according to the Indian Startup Ecosystem funding stage
    """
    classification = {"Ideation": ["Pre-seed", "Pre-Seed", "Pre seed Round", "Early seed", "Pre-seed Round", "Pre seed round"],
                "Validation": ["Angel", "Seed", "Seed A", "Seed Funding", "Seed round", "Seed Round", "Seed Round & Series A", "Grant",
                "Seed Investment", "Angel Round", "Seed fund", "Seed funding", "Seed+",  "Corporate Round"],
                "Early Traction": ["Series A", "Series A-1", "Series A+", "Series A2", "Seies A", "Pre-Series A", "Pre-series A", "Pre- series A",
                "Venture - Series Unknown", "Pre series A1", "Pre-series A1", "Post series A", "Pre Series A", "Pre series A", "Pre-series"],
                "Scaling": ["Series B+", "Series B", "Series B2", "Series B3", "Series C", "Series C, D", "Series D", "Series D1", "Series E",
                "Series F", "Series F1", "Series F2", "Series G", "Series H", "Series I", "Mid series", "Bridge", "Edge", "Pre-series C",
                "Pre series C", "Pre-series B", "Pre-Series B", "Pre series B", "Private Equity (PE)", "PE", "Private Equity"],
                "Exit Options": ["Post-IPO Debt", "Post-IPO Equity", "Debt", "Debt Financing", "Bridge Round"],
                "Others": ["Fresh funding", "Funding Round", "Non-equity Assistance", "Secondary Market", "Undisclosed",
                           "https://docs.google.com/spreadsheets/d/1x9ziNeaz6auNChIHnMI8U6kS7knTr3byy_YBGfQaoUA/edit#gid=1861303593"]}
    
    # Put stages under class
    for classes, stage in classification.items():
        if value in stage:
            value = classes
    return value

# Apply function to column
df_combined["stage"] = df_combined["stage"].apply(stage_classification)
# Preview column
df_combined["stage"].unique()

array([None, 'Ideation', 'Validation', 'Early Traction', 'Scaling',
       'Exit Options', nan, 'Others'], dtype=object)

### `Column: sector`

In [40]:
df_combined["sector"].unique()

array(['AgriTech', 'EdTech', 'Hygiene management', 'Escrow',
       'Networking platform', 'FinTech', 'Crowdsourcing',
       'Food & Bevarages', 'HealthTech', 'Fashion startup',
       'Food Industry', 'Food Delivery', 'Virtual auditing startup',
       'E-commerce', 'Gaming', 'Work fulfillment', 'AI startup',
       'Telecommunication', 'Logistics', 'Tech Startup', 'Sports',
       'Retail', 'Medtech', 'Tyre management', 'Cloud company',
       'Software company', 'Venture capitalist', 'Renewable player',
       'IoT startup', 'SaaS startup', 'Aero company', 'Marketing company',
       'Retail startup', 'Co-working Startup', 'Finance company',
       'Tech company', 'Solar Monitoring Company',
       'Video sharing platform', 'Gaming startup',
       'Video streaming platform', 'Consumer appliances',
       'Blockchain startup', 'Conversational AI platform', 'Real Estate',
       'SaaS platform', 'AI platform', 'Fusion beverages', 'HR Tech',
       'Job portal', 'Hospitality', 'Digit

In [39]:
# import pandas as pd
# import re

# # Sample DataFrame with mixed single-entry and three-entry values
# data = {
#     'column': ['single', 'value1,value2,value3', 'another1,another2,another3', 'one', 'test1,test2,test3']
# }
# df = pd.DataFrame(data)

# # Regular expression to capture the second value separated by commas
# regex = r'^[^,]*,([^,]*),[^,]*$'

# # Function to extract second value if it exists, otherwise return the original single-entry value
# def extract_second_or_single(value):
#     if ',' in value:
#         match = re.match(regex, value)
#         if match:
#             return match.group(1)
#     return value

# # Apply the function to the column
# df['extracted_value'] = df['column'].apply(extract_second_or_single)

# # Display the DataFrame
# print(df)


## Hypothesis Testing